# Spark Analysis

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum, avg, count, desc, when
import warnings
warnings.filterwarnings('ignore')

## spark setup

In [2]:
spark = SparkSession.builder \
    .appName("StockPortfolioAnalysis") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

print(f"✅ Spark Session created successfully")
print(f"Spark Version: {spark.version}")
print(f"Master: {spark.sparkContext.master}")

✅ Spark Session created successfully
Spark Version: 3.5.3
Master: spark://spark-master:7077


-----------------
## Data extraction

In [3]:
df = spark.read.csv(
    "/home/jovyan/output/FULL_STOCKS.csv",
    header=True,
    inferSchema=True
)

print(f"✅ Data loaded successfully")
print(f"Total rows: {df.count()}")
print(f"Total columns: {len(df.columns)}")

✅ Data loaded successfully
Total rows: 10000
Total columns: 16


In [4]:
print("="*80)
print("FIRST 10 RECORDS")
print("="*80)
df.show(10, truncate=False)

print("\nDataFrame Schema:")
df.printSchema()

FIRST 10 RECORDS
+--------------+----------+-----------+------------+----------------+--------+------------------+------------------+------------------+---------------------+--------+----------+----------+--------------------+------------+--------------+
|transaction_id|timestamp |customer_id|stock_ticker|transaction_type|quantity|average_trade_size|stock_price       |total_trade_amount|customer_account_type|day_name|is_weekend|is_holiday|stock_liquidity_tier|stock_sector|stock_industry|
+--------------+----------+-----------+------------+----------------+--------+------------------+------------------+------------------+---------------------+--------+----------+----------+--------------------+------------+--------------+
|6253          |2024-03-14|1006       |5           |0               |477     |267.17625         |71.65242981011856 |34178.209019426555|1                    |2       |0         |0         |High                |1           |2             |
|4685          |2023-12-04|4955

-----------
## Questions

### Q1. Total Trading Volume by Stock Ticker

In [5]:
print("="*80)
print("Q1: Total Trading Volume for Each Stock Ticker")
print("="*80)

q1_result = df.groupBy("stock_ticker") \
    .agg(spark_sum("quantity").alias("total_volume")) \
    .orderBy(desc("total_volume"))

q1_result.show(20, truncate=False)

# Save results
q1_result.coalesce(1).write.mode("overwrite") \
    .option("header", "true") \
    .csv("output/spark_results/q1_volume_by_ticker")

print("✅ Q1 completed and saved")

Q1: Total Trading Volume for Each Stock Ticker
+------------+------------+
|stock_ticker|total_volume|
+------------+------------+
|5           |611667      |
|11          |428057      |
|8           |314915      |
|17          |153811      |
|12          |152604      |
|9           |125862      |
|7           |104680      |
|19          |49724       |
|16          |35435       |
|18          |34229       |
|10          |32119       |
|6           |28486       |
|2           |8570        |
|4           |6997        |
|1           |4405        |
|0           |3557        |
|14          |3302        |
|15          |3210        |
|3           |2857        |
|13          |1512        |
+------------+------------+

✅ Q1 completed and saved


### Q2. verage Stock Price by Sector

In [6]:
print("="*80)
print("Q2: Average Stock Price by Sector")
print("="*80)

q2_result = df.groupBy("stock_sector") \
    .agg(avg("stock_price").alias("average_price")) \
    .orderBy(desc("average_price"))

q2_result.show(truncate=False)

# Save results
q2_result.coalesce(1).write.mode("overwrite") \
    .option("header", "true") \
    .csv("output/spark_results/q2_avg_price_by_sector")

print("✅ Q2 completed and saved")

Q2: Average Stock Price by Sector
+------------+------------------+
|stock_sector|average_price     |
+------------+------------------+
|0           |213.6248469077013 |
|4           |153.67922533016082|
|3           |152.00790316478043|
|1           |101.49595512796058|
|2           |79.92351314619137 |
+------------+------------------+

✅ Q2 completed and saved


### Q3. Weekend Transactions (Buy vs Sell)

In [7]:
print("="*80)
print("Q3: Buy vs Sell Transactions on Weekends")
print("="*80)

# Filter for weekend transactions
weekend_df = df.filter(col("is_weekend") == 1)

q3_result = weekend_df.groupBy("transaction_type") \
    .agg(count("transaction_id").alias("transaction_count")) \
    .orderBy(desc("transaction_count"))

q3_result.show(truncate=False)

total_weekend = weekend_df.count()
print(f"\nTotal weekend transactions: {total_weekend}")

# Save results
q3_result.coalesce(1).write.mode("overwrite") \
    .option("header", "true") \
    .csv("output/spark_results/q3_weekend_transactions")

print("✅ Q3 completed and saved")

Q3: Buy vs Sell Transactions on Weekends
+----------------+-----------------+
|transaction_type|transaction_count|
+----------------+-----------------+
+----------------+-----------------+


Total weekend transactions: 0
✅ Q3 completed and saved


### Q4. Customers with More Than 10 Transactions

In [8]:
print("="*80)
print("Q4: Customers with More Than 10 Transactions")
print("="*80)

q4_result = df.groupBy("customer_id") \
    .agg(count("transaction_id").alias("transaction_count")) \
    .filter(col("transaction_count") > 10) \
    .orderBy(desc("transaction_count"))

print(f"Total customers with >10 transactions: {q4_result.count()}")
print("\nTop 20 customers:")
q4_result.show(20, truncate=False)

# Save results
q4_result.coalesce(1).write.mode("overwrite") \
    .option("header", "true") \
    .csv("output/spark_results/q4_active_customers")

print("✅ Q4 completed and saved")

Q4: Customers with More Than 10 Transactions
Total customers with >10 transactions: 74

Top 20 customers:
+-----------+-----------------+
|customer_id|transaction_count|
+-----------+-----------------+
|4747       |1826             |
|4955       |897              |
|3588       |611              |
|3938       |444              |
|193        |419              |
|1513       |417              |
|4519       |284              |
|1157       |227              |
|845        |209              |
|4023       |189              |
|182        |162              |
|2426       |113              |
|227        |102              |
|1816       |86               |
|3339       |82               |
|1185       |73               |
|4700       |63               |
|474        |61               |
|3163       |47               |
|1494       |45               |
+-----------+-----------------+
only showing top 20 rows

✅ Q4 completed and saved


### Q5. Total Trade Amount by Day of Week

In [9]:
print("="*80)
print("Q5: Total Trade Amount per Day of Week (Highest to Lowest)")
print("="*80)

q5_result = df.groupBy("day_name") \
    .agg(spark_sum("total_trade_amount").alias("total_trade_amount")) \
    .orderBy(desc("total_trade_amount"))

q5_result.show(truncate=False)

# Save results
q5_result.coalesce(1).write.mode("overwrite") \
    .option("header", "true") \
    .csv("output/spark_results/q5_trade_by_day")

print("✅ Q5 completed and saved")

Q5: Total Trade Amount per Day of Week (Highest to Lowest)
+--------+--------------------+
|day_name|total_trade_amount  |
+--------+--------------------+
|2       |6.074148518912527E7 |
|4       |5.929508948897399E7 |
|1       |5.869949108159069E7 |
|0       |5.809298904268309E7 |
|3       |5.2834481894753695E7|
+--------+--------------------+

✅ Q5 completed and saved


---------------------
## Register DataFrame for SQL Queries

In [10]:
df.createOrReplaceTempView("stocks")
print("✅ DataFrame registered as 'stocks' view for SQL queries")

✅ DataFrame registered as 'stocks' view for SQL queries


### Q1. Top 5 Most Traded Stocks by Quantity

In [11]:
print("="*80)
print("SQL Q1: Top 5 Most Traded Stock Tickers by Total Quantity")
print("="*80)

sql_query_1 = """
    SELECT 
        stock_ticker,
        SUM(quantity) as total_quantity
    FROM stocks
    GROUP BY stock_ticker
    ORDER BY total_quantity DESC
    LIMIT 5
"""

sql_q1_result = spark.sql(sql_query_1)
sql_q1_result.show(truncate=False)

# Save results
sql_q1_result.coalesce(1).write.mode("overwrite") \
    .option("header", "true") \
    .csv("output/spark_results/sql_q1_top_5_stocks")

print("✅ SQL Q1 completed and saved")

SQL Q1: Top 5 Most Traded Stock Tickers by Total Quantity
+------------+--------------+
|stock_ticker|total_quantity|
+------------+--------------+
|5           |611667        |
|11          |428057        |
|8           |314915        |
|17          |153811        |
|12          |152604        |
+------------+--------------+

✅ SQL Q1 completed and saved


### Q2. Average Trade Amount by Account Type

In [12]:
print("="*80)
print("SQL Q2: Average Trade Amount by Customer Account Type")
print("="*80)

sql_query_2 = """
    SELECT 
        customer_account_type,
        AVG(total_trade_amount) as avg_trade_amount,
        COUNT(*) as transaction_count
    FROM stocks
    GROUP BY customer_account_type
    ORDER BY avg_trade_amount DESC
"""

sql_q2_result = spark.sql(sql_query_2)
sql_q2_result.show(truncate=False)

# Save results
sql_q2_result.coalesce(1).write.mode("overwrite") \
    .option("header", "true") \
    .csv("output/spark_results/sql_q2_avg_by_account")

print("✅ SQL Q2 completed and saved")

SQL Q2: Average Trade Amount by Customer Account Type
+---------------------+------------------+-----------------+
|customer_account_type|avg_trade_amount  |transaction_count|
+---------------------+------------------+-----------------+
|1                    |29250.822437240735|9113             |
|0                    |26043.733739066833|887              |
+---------------------+------------------+-----------------+

✅ SQL Q2 completed and saved


### Q3. Holiday vs Non-Holiday Transactions

In [13]:
print("="*80)
print("SQL Q3: Transactions During Holidays vs Non-Holidays")
print("="*80)

sql_query_3 = """
    SELECT 
        CASE 
            WHEN is_holiday = 1 THEN 'Holiday'
            ELSE 'Non-Holiday'
        END as period_type,
        COUNT(*) as transaction_count,
        SUM(total_trade_amount) as total_trade_amount
    FROM stocks
    GROUP BY is_holiday
    ORDER BY is_holiday DESC
"""

sql_q3_result = spark.sql(sql_query_3)
sql_q3_result.show(truncate=False)

# Save results
sql_q3_result.coalesce(1).write.mode("overwrite") \
    .option("header", "true") \
    .csv("output/spark_results/sql_q3_holiday_comparison")

print("✅ SQL Q3 completed and saved")

SQL Q3: Transactions During Holidays vs Non-Holidays
+-----------+-----------------+--------------------+
|period_type|transaction_count|total_trade_amount  |
+-----------+-----------------+--------------------+
|Holiday    |180              |5249307.697105034   |
|Non-Holiday|9820             |2.8441422900002223E8|
+-----------+-----------------+--------------------+

✅ SQL Q3 completed and saved


### Q4. Sectors with Highest Weekend Volume

In [14]:
print("="*80)
print("SQL Q4: Stock Sectors with Highest Weekend Trading Volume")
print("="*80)

sql_query_4 = """
    SELECT 
        stock_sector,
        SUM(quantity) as total_weekend_volume,
        COUNT(*) as weekend_transactions,
        SUM(total_trade_amount) as total_weekend_value
    FROM stocks
    WHERE is_weekend = 1
    GROUP BY stock_sector
    ORDER BY total_weekend_volume DESC
"""

sql_q4_result = spark.sql(sql_query_4)
sql_q4_result.show(truncate=False)

# Save results
sql_q4_result.coalesce(1).write.mode("overwrite") \
    .option("header", "true") \
    .csv("output/spark_results/sql_q4_weekend_sectors")

print("✅ SQL Q4 completed and saved")

SQL Q4: Stock Sectors with Highest Weekend Trading Volume
+------------+--------------------+--------------------+-------------------+
|stock_sector|total_weekend_volume|weekend_transactions|total_weekend_value|
+------------+--------------------+--------------------+-------------------+
+------------+--------------------+--------------------+-------------------+

✅ SQL Q4 completed and saved


### Q5. Buy vs Sell by Liquidity Tier

In [15]:
print("="*80)
print("SQL Q5: Total Buy vs Sell Amount by Stock Liquidity Tier")
print("="*80)

sql_query_5 = """
    SELECT 
        stock_liquidity_tier,
        SUM(CASE WHEN transaction_type = 'BUY' THEN total_trade_amount ELSE 0 END) as total_buy_amount,
        SUM(CASE WHEN transaction_type = 'SELL' THEN total_trade_amount ELSE 0 END) as total_sell_amount,
        SUM(total_trade_amount) as total_amount,
        COUNT(CASE WHEN transaction_type = 'BUY' THEN 1 END) as buy_count,
        COUNT(CASE WHEN transaction_type = 'SELL' THEN 1 END) as sell_count
    FROM stocks
    GROUP BY stock_liquidity_tier
    ORDER BY total_amount DESC
"""

sql_q5_result = spark.sql(sql_query_5)
sql_q5_result.show(truncate=False)

# Save results
sql_q5_result.coalesce(1).write.mode("overwrite") \
    .option("header", "true") \
    .csv("output/spark_results/sql_q5_liquidity_analysis")

print("✅ SQL Q5 completed and saved")

SQL Q5: Total Buy vs Sell Amount by Stock Liquidity Tier
+--------------------+----------------+-----------------+--------------------+---------+----------+
|stock_liquidity_tier|total_buy_amount|total_sell_amount|total_amount        |buy_count|sell_count|
+--------------------+----------------+-----------------+--------------------+---------+----------+
|High                |0.0             |0.0              |2.2561455653055984E8|0        |0         |
|Mid                 |0.0             |0.0              |6.107445517901835E7 |0        |0         |
|Low                 |0.0             |0.0              |2974524.9875485194  |0        |0         |
+--------------------+----------------+-----------------+--------------------+---------+----------+

✅ SQL Q5 completed and saved


In [16]:
print("\n" + "="*80)
print("ANALYSIS SUMMARY")
print("="*80)
print("✅ All 5 Spark DataFrame questions completed")
print("✅ All 5 Spark SQL questions completed")
print("✅ Results saved to output/spark_results/")
print("="*80)

spark.stop()
print("\n✅ Spark session stopped")


ANALYSIS SUMMARY
✅ All 5 Spark DataFrame questions completed
✅ All 5 Spark SQL questions completed
✅ Results saved to output/spark_results/

✅ Spark session stopped
